In [1]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from torchvision.datasets import ImageFolder
import os
import random
import itertools
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import roc_auc_score
!pip install efficientnet_pytorch
import efficientnet_pytorch

In [2]:
path = '../input/melanoma-classification/Melanoma New/train'
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

dataset = ImageFolder(path, transform = transform)

batch_size = 32
validation_split = .2
shuffle_dataset = True
random_seed= 42

dataset_size = len(dataset)
indices = list(range(dataset_size))
split = int(np.floor(validation_split * dataset_size))
if shuffle_dataset:
    np.random.seed(random_seed)
    np.random.shuffle(indices)
train_indices, val_indices = indices[split:], indices[:split]

train_sampler = SubsetRandomSampler(train_indices)
valid_sampler = SubsetRandomSampler(val_indices)

trainloader = DataLoader(
    dataset, 
    batch_size=batch_size,
    sampler=train_sampler)

validloader = DataLoader(
    dataset, 
    batch_size=batch_size,
    sampler=valid_sampler)

In [3]:
len(trainloader), len(validloader)

In [4]:
from torchvision.utils import make_grid

for images, labels in trainloader:
    fig, ax = plt.subplots(figsize = (10, 10))
    ax.set_xticks([])
    ax.set_yticks([])
    images = (images*0.5)+0.5
    ax.imshow(make_grid(images, 8).permute(1,2,0))
    break

In [5]:
class EfficientNet(nn.Module):
    def __init__(self):
        super(EfficientNet, self).__init__()
        self.base_model = efficientnet_pytorch.EfficientNet.from_pretrained('efficientnet-b4')
        self.base_model._fc = nn.Linear(in_features=1792, out_features=1, bias=True)
        
    def forward(self, image, targets):
        out = self.base_model(image)
        out = torch.sigmoid(out)
        loss = nn.BCELoss()(out, targets.view(-1, 1).type_as(out))
        return out, loss

In [6]:
model = EfficientNet()

In [7]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer,
        patience=3,
        threshold=0.001,
        mode="max"
    )

In [8]:
cuda = torch.cuda.is_available()
print(f'cuda: {cuda}')
if cuda:
    model = model.cuda()
    for state in optimizer.state.values():
        for k, v in state.items():
            if isinstance(v, torch.Tensor):
                state[k] = v.cuda()

In [14]:
n_epochs = 10
valid_loss_min = np.Inf
roc_auc_min = np.Inf
val_loss = []
val_acc = []
val_roc_auc = []
train_loss = []
train_acc = []
train_roc_auc = []
total_step = len(trainloader)
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
for epoch in range(1, n_epochs+1):
    running_loss = 0.0
    correct = 0
    total= 0
    roc_auc = 0
    print(f'Epoch {epoch}\n')
    for batch_idx, (data_, target_) in enumerate(trainloader):
        data_, target_ = data_.type(Tensor), target_.type(Tensor)# on GPU
        # zero the parameter gradients
        optimizer.zero_grad()
        # forward + backward + optimize
        y_pred, loss = model(data_, target_)
        loss.backward()
        optimizer.step()
        # print statistics
        running_loss += loss.item()
        auc_roc_batch = roc_auc_score(target_.cpu().detach().numpy(), y_pred.cpu().detach().numpy())
        y_pred = (y_pred>0.5).float().squeeze(1)
        correct += torch.sum(y_pred==target_).item()
        total += target_.size(0)
        roc_auc += auc_roc_batch
        if (batch_idx) % 100 == 0:
            print ('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}' 
                   .format(epoch, n_epochs, batch_idx, total_step, loss.item()))
    train_acc.append(100 * correct / total)
    train_loss.append(running_loss/total_step)
    train_roc_auc.append(roc_auc/total_step)
    print(f'\ntrain loss: {np.mean(train_loss):.4f}, train acc: {(100 * correct / total):.4f}, train roc auc: {(roc_auc/total_step):.4f}')
    batch_loss = 0
    total_t = 0
    correct_t = 0
    roc_auc_t = 0
    with torch.no_grad():
        model.eval()
        for data_t, target_t in (validloader):
            data_t, target_t = data_t.type(Tensor), target_t.type(Tensor) # on GPU
            y_pred_t, loss_t = model(data_t, target_t)
            batch_loss += loss_t.item()
            auc_roc_batch = roc_auc_score(target_t.cpu().numpy(), y_pred_t.cpu().numpy())
            y_pred_t = (y_pred_t>0.5).float().squeeze(1)
            correct_t += torch.sum(y_pred_t==target_t).item()
            total_t += target_t.size(0)
            roc_auc_t += auc_roc_batch
        val_acc.append(100 * correct_t / total_t)
        val_loss.append(batch_loss/len(validloader))
        val_roc_auc.append(roc_auc_t/len(validloader))
#         network_learned = batch_loss < valid_loss_min
        network_learned = roc_auc_t < roc_auc_min
        print(f'validation loss: {np.mean(val_loss):.4f}, validation acc: {(100 * correct_t / total_t):.4f}, validation roc auc: {(roc_auc_t/len(validloader)):.4f}\n')
        # Saving the best weight
        if network_learned:
#             valid_loss_min = batch_loss
            roc_auc_min = roc_auc_t
            torch.save(model.state_dict(), './model_classification.pth')
            print('Detected network improvement, saving current model')
    model.train()